In [40]:
import pandas as pd

In [41]:
df = pd.read_csv(r"..\outputs\emissions.csv", sep=";")
df.head()

,timestep_time,vehicle_CO,vehicle_CO2,vehicle_HC,vehicle_NOx,vehicle_PMx,vehicle_angle,vehicle_eclass,vehicle_electricity,vehicle_fuel,vehicle_id,vehicle_lane,vehicle_noise,vehicle_pos,vehicle_route,vehicle_speed,vehicle_type,vehicle_waiting,vehicle_x,vehicle_y
0,0.0,112.58,2298.06,0.56,0.96,0.09,90.0,HBEFA3/PC_G_EU6,0.0,733.06,flow_0.0,A4B4_0,55.94,4.60,r_upper,0.00,car,0.0,7.80,398.4
1,1.0,102.77,3237.74,0.54,1.38,0.09,90.0,HBEFA3/PC_G_EU6,0.0,1032.87,flow_0.0,A4B4_0,66.54,6.82,r_upper,2.22,car,0.0,10.02,398.4
2,2.0,101.60,4718.86,0.58,2.05,0.12,90.0,HBEFA3/PC_G_EU6,0.0,1505.40,flow_0.0,A4B4_0,68.89,11.55,r_upper,4.73,car,0.0,14.75,398.4
3,3.0,97.84,5741.30,0.59,2.51,0.13,90.0,HBEFA3/PC_G_EU6,0.0,1831.62,flow_0.0,A4B4_0,69.16,18.62,r_upper,7.07,car,0.0,21.82,398.4
4,4.0,99.44,6887.45,0.63,3.02,0.15,90.0,HBEFA3/PC_G_EU6,0.0,2197.29,flow_0.0,A4B4_0,70.00,27.97,r_upper,9.36,car,0.0,31.17,398.4


In [42]:
# Order by vehicle_id and timestep

# Creates a new column 'car_id' as the integer after the first dot, avoids recurrent flagging
df["car_id"] = df["vehicle_id"].str.split(".", n=1).str[1].astype(int)

# Sort by car_id and timestep_time
df = df.sort_values(by=["car_id", "timestep_time"], ascending=[True, True])

df.head()
df.head()

,timestep_time,vehicle_CO,vehicle_CO2,vehicle_HC,vehicle_NOx,vehicle_PMx,vehicle_angle,vehicle_eclass,vehicle_electricity,vehicle_fuel,...,vehicle_lane,vehicle_noise,vehicle_pos,vehicle_route,vehicle_speed,vehicle_type,vehicle_waiting,vehicle_x,vehicle_y,car_id
0,0.0,112.58,2298.06,0.56,0.96,0.09,90.0,HBEFA3/PC_G_EU6,0.0,733.06,...,A4B4_0,55.94,4.60,r_upper,0.00,car,0.0,7.80,398.4,0
1,1.0,102.77,3237.74,0.54,1.38,0.09,90.0,HBEFA3/PC_G_EU6,0.0,1032.87,...,A4B4_0,66.54,6.82,r_upper,2.22,car,0.0,10.02,398.4,0
2,2.0,101.60,4718.86,0.58,2.05,0.12,90.0,HBEFA3/PC_G_EU6,0.0,1505.40,...,A4B4_0,68.89,11.55,r_upper,4.73,car,0.0,14.75,398.4,0
3,3.0,97.84,5741.30,0.59,2.51,0.13,90.0,HBEFA3/PC_G_EU6,0.0,1831.62,...,A4B4_0,69.16,18.62,r_upper,7.07,car,0.0,21.82,398.4,0
4,4.0,99.44,6887.45,0.63,3.02,0.15,90.0,HBEFA3/PC_G_EU6,0.0,2197.29,...,A4B4_0,70.00,27.97,r_upper,9.36,car,0.0,31.17,398.4,0


In [ ]:
car_ids = list(sorted(set(df["car_id"])))

missing_ids = [i for i in range(car_ids[0], car_ids[-1] + 1) if i not in car_ids]

missing_info = [{"missing_id": m,
                 "prev_id": m-1 if m-1 in car_ids else None,
                 "next_id": m+1 if m+1 in car_ids else None} for m in missing_ids]

print("Missing IDs:", missing_ids)
print("Details:", missing_info)

Missing IDs: []
Details: []


In [ ]:
# Group by vehicle_id and compute average CO2 per vehicle
avg_df = df.groupby("car_id", as_index=False).agg({"vehicle_CO2": "mean"}).rename(columns={"vehicle_CO2": "avg_co2"})
avg_df.head(20)

,car_id,avg_co2
0,0,2883.211356
1,1,2607.713651
2,2,2855.893167
3,3,2844.316230
4,4,2715.087500
5,5,2360.326197
6,6,2652.302576
7,7,2510.529231
8,8,2540.325238
9,9,2361.149275
